In [1]:
import pandas as pd
import numpy 

import sklearn

import scipy



In [50]:
pheno = pd.read_table("pheno.txt")
data = pd.read_table("normalized_data.txt")

pheno.head()


,ParticipantID,Phenotype,Sample_Source,PlateId,Sex,HybControlNormScale,RowCheck,NormScale_20,NormScale_0_005,NormScale_0_5,ANMLFractionUsed_20,ANMLFractionUsed_0_005,ANMLFractionUsed_0_5
0,C1,Control,Blood,P0023923,F,0.947480,PASS,0.769958,0.813128,0.818578,0.868,0.838,0.745
1,C2,Control,Blood,P0023895,F,0.891341,PASS,0.765640,0.967612,0.891220,0.874,0.900,0.790
2,C3,Control,Blood,P0023895,F,0.888196,PASS,0.412029,0.918688,0.810701,0.732,0.881,0.666
3,C4,Control,Blood,P0023923,F,0.856026,PASS,0.774548,0.866460,0.810048,0.879,0.875,0.777
4,C5,Control,Blood,P0023895,F,0.982222,PASS,0.665371,0.833492,0.820785,0.804,0.900,0.852


In [51]:
data.head()

metadata = data.iloc[:, :4]

metadata

,Molecule,Protein,Gene,UniProt
0,10000-28,CRBB2,CRYBB2,P43320
1,10001-7,c-Raf,RAF1,P04049
2,10003-15,ZNF41,ZNF41,P51814
3,10006-25,ELK1,ELK1,P19419
4,10008-43,GUC1A,GUCA1A,P43080
...,...,...,...,...
5279,9993-11,ZN264,ZNF264,O43296
5280,9994-217,ATP4B,ATP4B,P51164
5281,9995-6,DUT,DUT,P33316
5282,9997-12,UBXN4,UBXN4,Q92575


In [52]:
data = data.transpose()



In [53]:
data.columns = data.iloc[0]
data.drop(data.index[0:4], inplace=True)





In [54]:

data

Molecule,10000-28,10001-7,10003-15,10006-25,10008-43,10011-65,10012-5,10013-34,10014-31,10015-119,...,9981-18,9983-97,9984-12,9986-14,9989-12,9993-11,9994-217,9995-6,9997-12,9999-1
C1,474.1,217.7,116.2,629.5,499.3,2207.5,1914.9,520.3,789.4,433.0,...,553.9,454.7,658.0,1212.9,351.3,1021.0,13772.7,1317.7,3844.4,1150.0
C2,585.7,227.2,129.2,598.7,448.9,2753.0,1689.7,514.4,863.5,456.9,...,519.5,427.8,654.6,2005.7,414.7,965.0,8755.4,8086.5,3996.7,1241.5
C3,518.6,224.6,194.1,454.7,412.1,2342.1,1851.2,572.6,786.8,443.1,...,1370.0,438.3,734.4,1164.7,325.4,1092.7,12753.7,771.7,1861.9,1772.3
C4,594.9,227.8,106.9,623.2,514.7,2290.0,1930.0,529.2,817.2,484.4,...,542.0,473.1,923.7,1350.5,367.5,949.4,10523.2,1601.9,5932.3,1306.9
C5,587.9,191.4,229.1,589.2,453.6,4069.8,2181.8,597.3,925.6,403.7,...,681.9,522.1,794.4,2613.8,453.0,1022.6,1947.5,1230.7,4186.1,1206.3
C8,533.5,234.5,149.8,557.2,416.6,2343.9,1575.9,447.1,786.1,603.3,...,493.6,408.3,609.3,932.4,414.3,1005.1,15008.2,1252.0,5320.6,1844.7
C9,520.3,201.3,109.6,564.9,439.6,2715.8,1988.0,546.7,849.2,464.1,...,516.2,579.3,657.0,2153.7,399.2,1099.3,2130.1,1151.8,6285.4,1623.1
C11,542.8,211.3,113.4,504.1,445.7,2143.4,1748.9,528.5,856.1,447.3,...,501.6,458.2,663.9,3912.5,383.8,1074.2,5809.7,1769.6,7425.7,1405.2
C14,535.7,231.8,122.1,590.0,528.4,2240.5,1746.8,511.1,771.5,447.8,...,571.9,451.2,632.2,2773.5,380.8,1000.9,11847.8,2557.8,10910.7,1988.7
C15,435.5,264.6,117.6,595.9,479.9,2204.0,1534.6,487.6,766.6,491.7,...,605.6,437.1,579.1,2443.3,390.6,920.8,9978.9,2335.4,12262.3,2927.2
